<a href="https://colab.research.google.com/github/bryann54/semantic-matcing/blob/main/semantic_matching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
predicted = pd.read_csv("/content/predicted_matches.csv")

In [ ]:
predicted.head()

,source_1,source_2
0,101,201
1,102,202
2,103,203
3,104,204
4,105,205


In [ ]:
source1 = pd.read_csv("/content/source_1.csv")

In [ ]:
source1.head()

,id,name
0,0,"[horses, asses, mules, and, hinnies, live, pur..."
1,1,"[horses, live, purebred, breeding, animals]"
2,2,"[horses, live, other, than, purebred, breeding..."
3,4,"[horses, asses, mules, and, hinnies, live, oth..."
4,5,"[bovine, animals, live, purebred, breeding, an..."


In [ ]:
source1.isna().sum()

id      0
name    0
dtype: int64

In [ ]:
source2 = pd.read_csv("/content/source_2.csv")

In [ ]:
source2.shape

(48943, 2)

In [ ]:
source2.isna().sum()

id      0
name    0
dtype: int64

In [ ]:
matched = pd.read_csv("/content/matched_data.csv")

In [ ]:
matched.head()

,source_1,source_2
0,"Cereals, barley",barley
1,Other fresh or chilled potatoes,potatoes
2,"Fruit, edible; cherries, fresh",cherries
3,"Bran, sharps and other residues, of maize",maize bran
4,"Cereals, millet",millets


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import nltk 
import string
import re

In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
pucn = string.punctuation
stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
from nltk.stem import WordNetLemmatizer 
wn = nltk.WordNetLemmatizer()

def clean_text(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    url_free = url.sub(r'', text)
    alpha = ''.join([word for word in url_free if not word.isdigit()])
    sent = re.sub('\w*\d\w*', '', alpha)
    html=re.compile(r'<.*?>')
    no_html = html.sub(r'',sent)
    no_punct=[words for words in no_html if words not in pucn]
    words_wo_punct=''.join(no_punct)
    words_wo_punct = words_wo_punct.lower()
    split=re.split("\W+",words_wo_punct) 
    stopword_free=[word for word in split if word not in stop]
    lematized = [wn.lemmatize(word) for word in stopword_free]
    return lematized



In [ ]:
source1['name'] = source1['name'].apply(lambda x: clean_text(x))
source2['name'] = source2['name'].apply(lambda x: clean_text(x))

In [ ]:
source1.head()

,id,name
0,0,"[horse, ass, mule, hinny, live, purebred, bree..."
1,1,"[horse, live, purebred, breeding, animal]"
2,2,"[horse, live, purebred, breeding, animal]"
3,4,"[horse, ass, mule, hinny, live, purebred, bree..."
4,5,"[bovine, animal, live, purebred, breeding, ani..."


In [ ]:
from gensim.models import Word2Vec

In [ ]:
model = Word2Vec(source1['name'], min_count=1)

In [ ]:
print(model)

Word2Vec(vocab=12326, size=100, alpha=0.025)


In [ ]:
print(model['horse'])

[-0.00210819 -0.03070619  0.04064171 -0.01135551 -0.03525279 -0.01598325
 -0.01416998 -0.0445454  -0.00504639  0.01669582  0.00848868  0.0004968
 -0.0272349  -0.01615568 -0.00779464 -0.03252409  0.00916708 -0.00889871
  0.00194699  0.02855228  0.0027424  -0.0241506   0.02691095  0.00074758
 -0.00763346  0.00020557 -0.01430524 -0.04566707  0.00236054 -0.01353863
 -0.02288015  0.00999316  0.00689493  0.02356789  0.00237263  0.00299617
  0.04745952  0.03393076  0.01948466  0.01536632  0.00742781 -0.02235918
  0.03186772 -0.03167711  0.00663115 -0.00817674 -0.0033329  -0.00757602
 -0.01457325 -0.009231    0.02286532 -0.02947263 -0.0121138   0.03252088
  0.00492774  0.0198931  -0.01815822 -0.01137419 -0.01460456  0.00464517
  0.01302869  0.02573903  0.01800429  0.022357    0.01058841 -0.00124676
 -0.01019798  0.01951634  0.00822378 -0.03334602  0.00532783 -0.00147966
  0.02127575 -0.05322452  0.01692115  0.00250747  0.01320926 -0.02087894
  0.04718561 -0.01672145  0.00234813 -0.00991231 -0.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


In [ ]:
print(model['hinny'])

[ 0.00422139 -0.01227573  0.01764791  0.00034326 -0.01629407 -0.00874473
 -0.00818593 -0.02595911  0.00162913  0.00574046  0.00545056  0.0002493
 -0.01148405 -0.00651508 -0.00499972 -0.01136229  0.00117635 -0.00929738
  0.00265042  0.01350558  0.00511675 -0.01359839  0.01873536 -0.00166594
 -0.00819542  0.00223507 -0.00594658 -0.02156582 -0.00309343 -0.00246728
 -0.01148972  0.00733937  0.00648797  0.01286277  0.00284809 -0.00380684
  0.02416452  0.01130559  0.01111605  0.01176987  0.00191555 -0.01312484
  0.01015293 -0.01157229  0.00433858 -0.00546224  0.00136206 -0.00789099
 -0.00980979 -0.00544598  0.01637366 -0.00985641 -0.00352838  0.01292606
 -0.00338171  0.01602148 -0.00524571 -0.00627463 -0.00836444  0.00245436
  0.01047637  0.00907274  0.01007619  0.01366434  0.00260414 -0.00401073
 -0.00499049  0.01211829 -0.00294717 -0.01531605  0.00286098  0.00434
  0.0111418  -0.02745815  0.00977731 -0.00328274  0.00744116 -0.01386063
  0.0266594  -0.00500234 -0.0011083  -0.00869095  0.002

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


In [ ]:
model.most_similar("hinny")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('prepared', 0.9642633199691772),
 ('others', 0.9640843272209167),
 ('otherwise', 0.9638186693191528),
 ('bean', 0.9635974168777466),
 ('except', 0.9630317687988281),
 ('preparation', 0.9629487991333008),
 ('bovine', 0.9628778696060181),
 ('leather', 0.9627896547317505),
 ('whole', 0.9624342918395996),
 ('skin', 0.9624214768409729)]

In [ ]:
import itertools
import csv

In [ ]:
# for word1,word2 in itertools.zip_longest(name1,name2):
#   for word in word1:
#     spac = nlp(word)
#     for wrd in word2:
#       spec = nlp(wrd)
#       print(spac.text, spec.text, spac.similarity(spec))

In [ ]:
# define lists
offices = ['Atlanta', 'Boston', 'New York', 'Miami']
employees = [100,120,140,150]

#define list of lists.
my_list = [offices, employees]

#---------replace with method 2 as needed----------
# create DataFrame
hr_df = pd.DataFrame(my_list)

# Transpose the data and add column names
hr_df = hr_df.T
hr_df.columns = ['office', 'employees']

#export to csv format
hr_df.to_csv('hr.csv', index = False)


In [ ]:
hr_df

,office,employees
0,Atlanta,100
1,Boston,120
2,New York,140
3,Miami,150
